## Import libraries

In [1]:
import re
import os
import gc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from bs4 import BeautifulSoup


import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input, Concatenate, Average, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping, ModelCheckpoint

import transformers
from transformers import TFAutoModel, AutoTokenizer
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

import warnings
warnings.filterwarnings("ignore")

C:\Anaconda3\envs\Machine_Learning\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Anaconda3\envs\Machine_Learning\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Anaconda3\envs\Machine_Learning\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Anaconda3\envs\Machine_Learning\lib\site-packages\numpy\.libs\libopenblas.wcdjnk7yvmpzq2me2zzhjjrj3jikndb7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


[Kaggle](https://www.kaggle.com/c/jigsaw-multilingual-toxic-comment-classification/data) 

## このデータで何を予測するのか
- コメントが有毒なコメントである確率を予測する。有毒なコメントは1.0。良性で毒性のないコメントは0.0。
- テストセットでは、すべてのコメントが1.0または、0.0のいずれかに分類されます。

## ファイル
- jigsaw-toxic-comment-train.csv-最初のコンテストのデータ。データセットは、ウィキペディアのトークページ編集からの英語のコメントで構成されています。
- jigsaw-unintended-bias-train.csv - 2番目のコンテストのデータ。これは、Civil Commentsデータセットの拡張バージョンであり、さまざまな追加ラベルが付いています。
- sample_submission.csv-正しい形式のサンプル送信ファイル
- test.csv-英語以外のさまざまな言語のウィキペディアトークページからのコメント。
- test_labels.csv-テストデータのグラウンドトゥルースラベル（競争デッドリン後に追加されたデータ）
- validate.csv-英語以外のさまざまな言語のウィキペディアトークページからのコメント。
- jigsaw-toxic-comment-train-processed-seqlen128.csv -BERT用に前処理されたトレーニングデータ
- jigsaw-unintended-bias-train-processed-seqlen128.csv -BERT用に前処理されたトレーニングデータ
- 検証-処理済み-seqlen128.csv -BERT用に前処理された検証データ
- test-processed-seqlen128.csv -BERT用に前処理されたテストデータ
## 列
- id-各ファイル内の識別子。
- comment_text-分類されるコメントのテキスト。
- lang-コメントの言語。
- toxic-コメントがに分類されるかどうかtoxic。（には存在しませんtest.csv。）
### Train1
- severe_toxic-劇毒 	
- obscene-不適切
- threat-脅迫
- insult-侮辱
- identity_hate-アイデンティティーヘイト

# データ読み込み

In [2]:
train = pd.read_csv("D:\JupyterNotebook\jigsaw-multilingaual_Datasets\jigsaw-toxic-comment-train.csv")

valid = pd.read_csv('D:\JupyterNotebook\jigsaw-multilingaual_Datasets\\validation.csv')

test = pd.read_csv('D:\JupyterNotebook\jigsaw-multilingaual_Datasets\\test.csv')

sub = pd.read_csv('D:\JupyterNotebook\jigsaw-multilingaual_Datasets\\sample_submission.csv')

## Helper Functions

In [3]:
def text_preprocessing(text):
    
    def cleaning_text(text):
        text = str(text)
        text = re.sub(r'[0-9"]', '', text) # number
        text = re.sub(r'#[\S]+\b', '', text) # hash
        text = re.sub(r'@[\S]+\b', '', text) # mention
        text = re.sub(r'https?\S+', '', text) # link
        text = re.sub(r'\s+', ' ', text) # multiple white spaces
        
        return text.strip()
    
    def sub_text(text):
        template = re.compile(r'https?://\S+|www\.\S+') #Removes website links 
        text = template.sub(r'', text)
        
        text = BeautifulSoup(text, 'lxml').get_text() #Removes HTML tags
    
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"
                                   u"\U0001F300-\U0001F5FF"
                                   u"\U0001F680-\U0001F6FF"
                                   u"\U0001F1E0-\U0001F1FF"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251" "]+")
        text = emoji_pattern.sub(r'', text)
    
        text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
        text = re.sub(' +', ' ', text) #Remove Extra Spaces
        text = text.strip() # remove spaces at the beginning and at the end of string

        return text
    
    def text_split(text):
        split_line = text.split(' ')
        if(len(split_line) > 160):
            text = ' '.join(split_line[:160]) + ' ' + ' '.join(split_line[-32:])
            
        return text
    
    process_text = sub_text(text)
    process_text = text_split(process_text)
    
    return process_text

In [4]:
def tk_encode(texts, tokenizer, maxlen=512):
    """
    return_tensors : {"PyTorch" : 'pt'}, {"TensorFlow": 'tf'}, {"NumPy": 'np'} 
    """
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_mask=True, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True,
        return_tensors='tf'
    )
    
    return {
        "input_ids": enc_di['input_ids'],
        "attention_mask": enc_di['attention_mask']
    }

## TPU configs
- TPUの使用

In [5]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [6]:
# Configuration
MODEL = 'jplu/tf-xlm-roberta-large' # bert-base-uncased
AUTO = tf.data.experimental.AUTOTUNE
SEED = 42
EPOCHS_1 = 20
EPOCHS_2 = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192
SHUFFLE = 2048
VERBOSE = 1

## 変換実行

In [7]:
train.toxic = train.toxic.round().astype(int)
train = train.sample(frac=1, random_state=SEED)
valid = valid.sample(frac=1, random_state=SEED)

In [8]:
%%time
train['comment_text'] = train['comment_text'].apply(lambda x: text_preprocessing(x))
valid['comment_text'] = valid['comment_text'].apply(lambda x: text_preprocessing(x))
test['content'] = test['content'].apply(lambda x: text_preprocessing(x))

Wall time: 1min 38s


In [9]:
import psutil 
from termcolor import colored

# メモリ容量を取得
mem = psutil.virtual_memory() 
print(f"total_memory: {colored(mem.total, 'yellow')}, memory_used: {colored(mem.used, 'yellow')}, memory_available, {colored(mem.available, 'yellow')}")
print(f"memory_used_per: {((mem.used / mem.total) * 100) :.0f}%")
print(f"available_memory: {((mem.available / mem.total) * 100) :.0f}%")

total_memory: 12767760384, memory_used: 7262695424, memory_available, 5505064960
memory_used_per: 57%
available_memory: 43%


In [10]:
n_train_steps = train.shape[0] // (BATCH_SIZE * 8)
n_valid_steps = valid.shape[0] // (BATCH_SIZE)

#del train1
#gc.collect()

In [13]:
# First load the real tokenizer
tokenizer = AutoTokenizer.from_pretrained('jplu/tf-xlm-roberta-large') # bert-base-uncased

In [16]:
%%time 
x_train = tk_encode(list(train.comment_text.values), tokenizer, maxlen=MAX_LEN)
x_valid = tk_encode(list(valid.comment_text.values), tokenizer, maxlen=MAX_LEN)
x_test = tk_encode(list(test.content.values), tokenizer, maxlen=MAX_LEN)

y_train = train.toxic.values
y_valid = valid.toxic.values

del train, valid
gc.collect()

Wall time: 54.2 s


0

## Dataset objects
- データセットがメモリに入る程に小さければ、データセットのcache()メソッドを使ってRAMにキャッシュする事で、時間が大幅に短縮される。
- RAMへのキャッシングは、データをロード前処理したあと、シャッフル、リピート、バッチへの分割、プリフェッチ前に行う。

In [17]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(SHUFFLE)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)


valid_dataset = (
    tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)


test1_dataset = (
    tf.data.Dataset.from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

del x_train, x_valid, y_train, y_valid, x_test, 
gc.collect()

0

## Callbacks

In [18]:
class ValTrainRatioCustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.sf}".format(logs["val_loss"] / logs["loss"]))
        
custom_call = ValTrainRatioCustomCallback()

In [19]:
learning_rate_reducation = ReduceLROnPlateau(monitor="val_accuracy", patience=2, mode="max", verbose=0, factor=0.7, min_lr=1e-7)
early_stopping = EarlyStopping(monitor="val_accuracy", patience=5, mode="max", verbose=0, restore_best_weights=True)

In [20]:
callbacks_list = [learning_rate_reducation, early_stopping, custom_call]

## Build model
- クラス化する場合、内部の追跡やモデルの保存当で問題が発生する可能性があるので、どうしてもこの柔軟性が必要でない限りシーケンシャルAPIや関数型APIを使用した方が良い。
- 以下では関数型APIを使用して各層の接続を行っている。

In [21]:
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
    sequence_output = transformer({"input_ids": input_word_ids, "attention_mask": attention_mask})[0]

    avg_pool = GlobalAveragePooling1D()(sequence_output)
    max_pool = GlobalMaxPooling1D()(sequence_output)
    cls_token = Concatenate()([avg_pool, max_pool])
    
    samples = []
    for n in range(2):
        sample = Dropout(0.4)(cls_token)
        sample = Dense(1, activation='sigmoid', name=f'sample_{n}')(sample)
        samples.append(sample)
    
    out = Average(name='output')(samples)
    model = Model(inputs={
                "input_ids": input_word_ids,
                "attention_mask": attention_mask
                }, 
                outputs=out,
                name='XLM-R')
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])
    
    return model

## Load to TPU

In [22]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Downloading:   0%|          | 0.00/3.05G [00:00<?, ?B/s]

KeyboardInterrupt: 

## Run model

In [ ]:
model_history_1 = model.fit(
    train_dataset,
    steps_per_epoch=n_train_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS_1,
    callbacks=callbacks_list,
    verbose=VERBOSE
 )

## Blending

In [ ]:
#Blending
multi_sub = model.predict(test1_dataset, verbose=1)

sub['toxic'] = multi_sub*0.75

#Post-processing
sub.loc[test["lang"] == "es", "toxic"] *= 1.11
sub.loc[test["lang"] == "fr", "toxic"] *= 1.06
sub.loc[test["lang"] == "it", "toxic"] *= 0.93
sub.loc[test["lang"] == "pt", "toxic"] *= 0.92
sub.loc[test["lang"] == "tr", "toxic"] *= 0.94

# min-max normalize
sub.toxic -= sub.toxic.min()
sub.toxic /= (sub.toxic.max() - sub.toxic.min())
sub.toxic.hist(bins=100, log=False, alpha=1)

## Submission

In [ ]:
sub.to_csv('submission.csv', index=False)

In [34]:
import psutil 
from termcolor import colored

# メモリ容量を取得
mem = psutil.virtual_memory() 
print(f"total_memory: {colored(mem.total, 'yellow')}, memory_used: {colored(mem.used, 'yellow')}, memory_available, {colored(mem.available, 'yellow')}")
print(f"memory_used_per: {((mem.used / mem.total) * 100) :.0f}%")
print(f"available_memory: {((mem.available / mem.total) * 100) :.0f}%")

total_memory: 12767760384, memory_used: 7800610816, memory_available, 4967149568
memory_used_per: 61%
available_memory: 39%
